***GENERATED CODE FOR linearregressionforrealestate PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ParcelID', 'transformation_label': 'String Indexer'}], 'feature': 'ParcelID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '105-177-001', 'max': '294-568-031', 'missing': '0', 'distinct': '482'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ParcelID'}, {'feature_label': 'ParcelID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ParcelID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ApartmentUnitNumber', 'transformation_label': 'String Indexer'}], 'feature': 'ApartmentUnitNumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '120', 'mean': '145.54', 'stddev': '237.6', 'min': '000B', 'max': 'S422', 'missing': '380', 'distinct': '101'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ApartmentUnitNumber'}, {'feature_label': 'ApartmentUnitNumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ApartmentUnitNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreetNameAndWay', 'transformation_label': 'String Indexer'}], 'feature': 'StreetNameAndWay', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ADELAIDE ST', 'max': 'ZION ST', 'missing': '0', 'distinct': '232'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StreetNameAndWay'}, {'feature_label': 'StreetNameAndWay', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreetNameAndWay')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OwnerLastName', 'transformation_label': 'String Indexer'}], 'feature': 'OwnerLastName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '111 HUNGERFORD LLC', 'max': 'ZUBAIR', 'missing': '0', 'distinct': '379'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OwnerLastName'}, {'feature_label': 'OwnerLastName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OwnerLastName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OwnerFirstName', 'transformation_label': 'String Indexer'}], 'feature': 'OwnerFirstName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '318', 'mean': '', 'stddev': '', 'min': 'ADEBISI', 'max': 'YVONNE', 'missing': '182', 'distinct': '272'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OwnerFirstName'}, {'feature_label': 'OwnerFirstName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OwnerFirstName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryGrantor', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryGrantor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1047 ALBANY AVE LLC', 'max': 'ZARRILLI BENNY', 'missing': '0', 'distinct': '432'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryGrantor'}, {'feature_label': 'PrimaryGrantor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryGrantor')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SaleDate', 'transformation_label': 'String Indexer'}], 'feature': 'SaleDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/11/2021', 'max': '9/9/2022', 'missing': '0', 'distinct': '288'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SaleDate'}, {'feature_label': 'SaleDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SaleDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LegalReference', 'transformation_label': 'String Indexer'}], 'feature': 'LegalReference', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '07631-0143', 'max': '08011-0105', 'missing': '0', 'distinct': '447'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LegalReference'}, {'feature_label': 'LegalReference', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LegalReference')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AssrLandUse', 'transformation_label': 'String Indexer'}], 'feature': 'AssrLandUse', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'APT CRDA', 'max': 'TWO FAMILY', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AssrLandUse'}, {'feature_label': 'AssrLandUse', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AssrLandUse')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run linearregressionforrealestateHooks.ipynb
try:
	#sourcePreExecutionHook()

	realestatesalesdays = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Real Estate Sales 730 Days.csv', 'filename': 'Real Estate Sales 730 Days.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(realestatesalesdays)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run linearregressionforrealestateHooks.ipynb
try:
	#transformationPreExecutionHook()

	linearregressionforrealestateautofe = TransformationMain.run(realestatesalesdays,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "PropertyID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "11400.72", "stddev": "6583.5", "min": "71", "max": "29427", "missing": "0"}, "updatedLabel": "PropertyID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrCompositeLandUseID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7.46", "stddev": "34.69", "min": "3", "max": "455", "missing": "0"}, "updatedLabel": "xrCompositeLandUseID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrBuildingTypeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "10.17", "stddev": "4.87", "min": "0", "max": "30", "missing": "0"}, "updatedLabel": "xrBuildingTypeID"}, {"transformationsData": [{"feature_label": "ParcelID", "transformation_label": "String Indexer"}], "feature": "ParcelID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "105-177-001", "max": "294-568-031", "missing": "0", "distinct": "482"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ParcelID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LocationStartNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "161.71", "stddev": "277.65", "min": "1", "max": "2718", "missing": "0"}, "updatedLabel": "LocationStartNumber"}, {"transformationsData": [{"feature_label": "ApartmentUnitNumber", "transformation_label": "String Indexer"}], "feature": "ApartmentUnitNumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "120", "mean": "145.54", "stddev": "237.6", "min": "000B", "max": "S422", "missing": "380", "distinct": "101"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ApartmentUnitNumber"}, {"transformationsData": [{"feature_label": "StreetNameAndWay", "transformation_label": "String Indexer"}], "feature": "StreetNameAndWay", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ADELAIDE ST", "max": "ZION ST", "missing": "0", "distinct": "232"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StreetNameAndWay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrPrimaryNeighborhoodID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "133.69", "stddev": "94.46", "min": "2", "max": "318", "missing": "0"}, "updatedLabel": "xrPrimaryNeighborhoodID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LandSF", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "380", "mean": "8079.78", "stddev": "5615.95", "min": "0", "max": "69696", "missing": "120"}, "updatedLabel": "LandSF"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalFinishedArea", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "485", "mean": "2284.12", "stddev": "1880.34", "min": "399.0", "max": "32692.0", "missing": "15"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalFinishedArea"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LivingUnits", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.73", "stddev": "1.42", "min": "0", "max": "26", "missing": "0"}, "updatedLabel": "LivingUnits"}, {"transformationsData": [{"feature_label": "OwnerLastName", "transformation_label": "String Indexer"}], "feature": "OwnerLastName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "111 HUNGERFORD LLC", "max": "ZUBAIR", "missing": "0", "distinct": "379"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OwnerLastName"}, {"transformationsData": [{"feature_label": "OwnerFirstName", "transformation_label": "String Indexer"}], "feature": "OwnerFirstName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "318", "mean": "", "stddev": "", "min": "ADEBISI", "max": "YVONNE", "missing": "182", "distinct": "272"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OwnerFirstName"}, {"transformationsData": [{"feature_label": "PrimaryGrantor", "transformation_label": "String Indexer"}], "feature": "PrimaryGrantor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1047 ALBANY AVE LLC", "max": "ZARRILLI BENNY", "missing": "0", "distinct": "432"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryGrantor"}, {"transformationsData": [{"feature_label": "SaleDate", "transformation_label": "String Indexer"}], "feature": "SaleDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/11/2021", "max": "9/9/2022", "missing": "0", "distinct": "288"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SaleDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SalePrice", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1339891.66", "stddev": "6393221.83", "min": "0", "max": "60500000", "missing": "0"}, "updatedLabel": "SalePrice"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalAppraisedValue", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "189797.36", "stddev": "169212.08", "min": "4400", "max": "3122600", "missing": "0"}, "updatedLabel": "TotalAppraisedValue"}, {"transformationsData": [{"feature_label": "LegalReference", "transformation_label": "String Indexer"}], "feature": "LegalReference", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "07631-0143", "max": "08011-0105", "missing": "0", "distinct": "447"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LegalReference"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrSalesValidityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "11.19", "stddev": "9.76", "min": "0", "max": "27", "missing": "0"}, "updatedLabel": "xrSalesValidityID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrDeedID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.12", "stddev": "2.55", "min": "1", "max": "19", "missing": "0"}, "updatedLabel": "xrDeedID"}, {"transformationsData": [{"feature_label": "AssrLandUse", "transformation_label": "String Indexer"}], "feature": "AssrLandUse", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "APT CRDA", "max": "TWO FAMILY", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AssrLandUse"}]}))

	#transformationPostExecutionHook(linearregressionforrealestateautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run linearregressionforrealestateHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(linearregressionforrealestateautofe, ["PropertyID", "xrCompositeLandUseID", "xrBuildingTypeID", "LocationStartNumber", "xrPrimaryNeighborhoodID", "LandSF", "TotalFinishedArea", "LivingUnits", "SalePrice", "TotalAppraisedValue", "xrSalesValidityID", "xrDeedID", "ParcelID_stringindexer", "ApartmentUnitNumber_stringindexer", "StreetNameAndWay_stringindexer", "OwnerLastName_stringindexer", "OwnerFirstName_stringindexer", "PrimaryGrantor_stringindexer", "LegalReference_stringindexer", "AssrLandUse_stringindexer"], "SaleDate_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

